# 쿼리 라우팅 1.
- 메타데이터 기반 라우터
- 노드레벨 라우팅 기능
- 메타데이터 기반 필터로 불필요한 노이즈 감소 (성능 up, 비용 down)

In [ ]:
!pip install llama_index openai qdrant_client llama-index-vector-stores-qdrant

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [6]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client
from qdrant_client import models
client = qdrant_client.QdrantClient(
    url="",
    api_key="",
)


In [3]:
from llama_index.core import VectorStoreIndex, StorageContext

In [4]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text=(
            "A bunch of scientists bring back dinosaurs and mayhem breaks"
            " loose"
        ),
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    TextNode(
        text=(
            "Leo DiCaprio gets lost in a dream within a dream within a dream"
            " within a ..."
        ),
        metadata={
            "year": 2010,
            "director": "Christopher Nolan",
            "rating": 8.2,
        },
    ),
    TextNode(
        text=(
            "Batman now serves as a under-the-hood hero in Gotham City..."
        ),
        metadata={
            "year": 2008,
            "director": "Christopher Nolan",
            "rating": 8.9,
        },
    ),
    TextNode(
        text=(
            "A psychologist / detective gets lost in a series of dreams within"
            " dreams within dreams and Inception reused the idea"
        ),
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    TextNode(
        text=(
            "A bunch of normal-sized women are supremely wholesome and some"
            " men pine after them"
        ),
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    TextNode(
        text="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
]

In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(temperature=0,model='gpt-4o-mini')

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name="metafilter_exercise")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

In [9]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo

vector_store_info = VectorStoreInfo(
    content_info="Brief summary of a movie",
    metadata_info=[
        MetadataInfo(
            name="genre",
            description="The genre of the movie",
            type="string or list[string]",
        ),
        MetadataInfo(
            name="year",
            description="The year the movie was released",
            type="integer",
        ),
        MetadataInfo(
            name="director",
            description="The name of the movie director",
            type="string",
        ),
        MetadataInfo(
            name="rating",
            description="A 1-10 rating for the movie",
            type="float",
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index, vector_store_info=vector_store_info, verbose=True
)

In [10]:
# 메타데이터 operand 활용하여 1개만 뜨는지 확인
retriever.retrieve(
    "What are 2 movies by Christopher Nolan were made before 2010?"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Movies directed by Christopher Nolan
Using query str: Movies directed by Christopher Nolan
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
Using query str: Movies directed by Christopher Nolan
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 

[NodeWithScore(node=TextNode(id_='e6d6bba9-c898-428d-a584-94fe82f36ee2', embedding=None, metadata={'year': 2008, 'director': 'Christopher Nolan', 'rating': 8.9}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Batman now serves as a under-the-hood hero in Gotham City...', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.5220654)]

In [11]:
# 두개 다 떠야할 경우 확인
retriever.retrieve(
    "What are 2 movies by Christopher Nolan were made before 2020?"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Movies directed by Christopher Nolan
Using query str: Movies directed by Christopher Nolan
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2020)]
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2020)]
Using query str: Movies directed by Christopher Nolan
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2020)]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 

[NodeWithScore(node=TextNode(id_='df7ebd00-1ddd-470a-a2d7-e6248e9d599b', embedding=None, metadata={'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.53621125),
 NodeWithScore(node=TextNode(id_='e6d6bba9-c898-428d-a584-94fe82f36ee2', embedding=None, metadata={'year': 2008, 'director': 'Christopher Nolan', 'rating': 8.9}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Batman now serves as a under-the-hood hero in Gotham City...', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='

In [12]:
# 한개 떠야할 경우 확인
retriever.retrieve(
    "What are the movies by Satoshi Kon before 2009?"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Movies directed by Satoshi Kon
Using query str: Movies directed by Satoshi Kon
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Satoshi Kon'), ('year', '<', '2009')]
Using filters: [('director', '==', 'Satoshi Kon'), ('year', '<', '2009')]
Using query str: Movies directed by Satoshi Kon
Using filters: [('director', '==', 'Satoshi Kon'), ('year', '<', '2009')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Reques

[NodeWithScore(node=TextNode(id_='93000df8-9a77-4e77-ab66-1bbfb1ebb0e5', embedding=None, metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.5448681)]

In [13]:
# 아무것도 안떠야 하는 경우 확인
retriever.retrieve("Has Andrei Tarkovsky directed any science fiction movies")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: science fiction movies directed by Andrei Tarkovsky
Using query str: science fiction movies directed by Andrei Tarkovsky
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
Using query str: science fiction movies directed by Andrei Tarkovsky
Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddin

[]

In [14]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
# 쿼리 엔진 붙히기
response_synthesizer = get_response_synthesizer()

metadata_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)


In [15]:
metadata_query_engine.query("What was the rating of the movie by Christopher Nolan that were made before 2010?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Rating of movies directed by Christopher Nolan
Using query str: Rating of movies directed by Christopher Nolan
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
Using query str: Rating of movies directed by Christopher Nolan
Using filters: [('director', '==', 'Christopher Nolan'), ('year', '<', 2010)]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.opena

Response(response='The rating of the movie by Christopher Nolan made before 2010 is 8.9.', source_nodes=[NodeWithScore(node=TextNode(id_='e6d6bba9-c898-428d-a584-94fe82f36ee2', embedding=None, metadata={'year': 2008, 'director': 'Christopher Nolan', 'rating': 8.9}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Batman now serves as a under-the-hood hero in Gotham City...', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.5815091)], metadata={'e6d6bba9-c898-428d-a584-94fe82f36ee2': {'year': 2008, 'director': 'Christopher Nolan', 'rating': 8.9}})

In [16]:
metadata_query_engine.query("Has Andrei Tarkovsky directed any science fiction movies?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: science fiction movies directed by Andrei Tarkovsky
Using query str: science fiction movies directed by Andrei Tarkovsky
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
Using query str: science fiction movies directed by Andrei Tarkovsky
Using filters: [('director', '==', 'Andrei Tarkovsky'), ('genre', '==', 'science fiction')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddin

Response(response='Empty Response', source_nodes=[], metadata=None)

In [17]:
# Naive RAG
query_engine = index.as_query_engine()

In [18]:
query_engine.query("What was the rating of the movie by Christopher Nolan that were made before 2010?")
# 메타데이터 필터 없이는 무조건 top-2로 retrieve되어 노이즈로써 컨텍스트 참고하게 됨.

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='The rating of the movie by Christopher Nolan that was made before 2010 is 8.9.', source_nodes=[NodeWithScore(node=TextNode(id_='df7ebd00-1ddd-470a-a2d7-e6248e9d599b', embedding=None, metadata={'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.5586288), NodeWithScore(node=TextNode(id_='e6d6bba9-c898-428d-a584-94fe82f36ee2', embedding=None, metadata={'year': 2008, 'director': 'Christopher Nolan', 'rating': 8.9}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Batman now serves as a under-the-ho

In [19]:
query_engine.query("Has Andrei Tarkovsky directed any science fiction movies?")
# 메타데이터 필터 없이는 무조건 top-2로 retrieve되어 노이즈로써 컨텍스트 참고하게 됨.

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='There is no information provided regarding Andrei Tarkovsky or his filmography in the context. Therefore, it cannot be determined if he has directed any science fiction movies.', source_nodes=[NodeWithScore(node=TextNode(id_='93000df8-9a77-4e77-ab66-1bbfb1ebb0e5', embedding=None, metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.30951488), NodeWithScore(node=TextNode(id_='24878131-4641-499c-ae2b-655b86a75ecb', embedding=None, metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_k

# Do It Yourself

In [20]:
# 예시 2
nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
            "theme": "Fiction",
            "year": 2010,
        },
    ),
    TextNode(
        text="To Kill a Mockingbird",
        metadata={
            "author": "Harper Lee",
            "theme": "Fiction",
            "year": 1960,
        },
    ),
    TextNode(
        text="1984",
        metadata={
            "author": "George Orwell",
            "theme": "Totalitarianism",
            "year": 1949,
        },
    ),
    TextNode(
        text="The Great Gatsby",
        metadata={
            "author": "F. Scott Fitzgerald",
            "theme": "The American Dream",
            "year": 1925,
        },
    ),
    TextNode(
        text="Harry Potter and the Sorcerer's Stone",
        metadata={
            "author": "J.K. Rowling",
            "theme": "Fiction",
            "year": 1997,
        },
    ),
]

In [21]:
vector_store = QdrantVectorStore(client=client, collection_name="metafilter_exercise2")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index2 = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:httpx:HTTP Request: GET https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/exists "HTTP/1.1 200 OK"
HTTP Request: GET https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2 "HTTP/1.1 200 OK"
HTTP Request: PUT https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/index?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT https

In [22]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="famous books and movies",
    metadata_info=[
        MetadataInfo(
            name="director",
            type="str",
            description=("Name of the director"),
        ),
        MetadataInfo(
            name="theme",
            type="str",
            description=("Theme of the book/movie"),
        ),
        MetadataInfo(
            name="year",
            type="int",
            description=("Year of the book/movie"),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index2,
    vector_store_info=vector_store_info,
    verbose=True,
)

In [23]:
nodes = retriever.retrieve(
    "Tell me about some books/movies after the year 2000"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: books or movies
Using query str: books or movies
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('year', '>', '2000')]
Using filters: [('year', '>', '2000')]
Using query str: books or movies
Using filters: [('year', '>', '2000')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/points/search "HTTP/1.1 200

In [24]:
for node in nodes:
    print(node.text)
    print(node.metadata)

Inception
{'director': 'Christopher Nolan', 'theme': 'Fiction', 'year': 2010}


In [25]:
nodes = retriever.retrieve("Tell me about some books that are Fiction")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: books that are Fiction
Using query str: books that are Fiction
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: []
Using filters: []
Using query str: books that are Fiction
Using filters: []
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://2dbee40a-5

In [26]:
for node in nodes:
    print(node.text)
    print(node.metadata)

Harry Potter and the Sorcerer's Stone
{'author': 'J.K. Rowling', 'theme': 'Fiction', 'year': 1997}
To Kill a Mockingbird
{'author': 'Harper Lee', 'theme': 'Fiction', 'year': 1960}


In [27]:
nodes = retriever.retrieve("Tell me about some books that are mafia-themed")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: mafia-themed books
Using query str: mafia-themed books
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: []
Using filters: []
Using query str: mafia-themed books
Using filters: []
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_exercise2/points/search "HTTP/1.1 200 OK"
HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5

# 개선 방안
- 프롬프트에서 기대하는 자연어 -> 필터가 자연스럽게 연결 안될때는 메타필터 변환 프롬프트를 강화해 주면 됨

In [28]:
from llama_index.core.prompts import display_prompt_dict
from llama_index.core import PromptTemplate
prompts_dict = retriever.get_prompts()
display_prompt_dict(prompts_dict)
prompts_dict["prompt"].template_vars

**Prompt Key**: prompt<br>**Text:** <br>

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

{schema_str}

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters take into account the descriptions of attributes.
Make sure that filters are only used as needed. If there are no filters that should be applied return [] for the filter value.
If the user's query explicitly mentions number of documents to retrieve, set top_k to that number, otherwise do not set top_k.

<< Example 1. >>
Data Source:
```json
{{
    "metadata_info": [
        {{
            "name": "artist",
            "type": "str",
            "description": "Name of the song artist"
    

<br><br>

['schema_str', 'info_str', 'query_str']

In [29]:
# 프롬프트 템플릿 수정

prompt_tmpl_str = """\
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

{schema_str}

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters take into account the descriptions of attributes.
Make sure that filters are only used as needed. If there are no filters that should be applied return [] for the filter value.
If the user's query explicitly mentions number of documents to retrieve, set top_k to that number, otherwise do not set top_k.
Do NOT EVER infer a null value for a filter. This will break the downstream program. Instead, don't include the filter.

<< Example 1. >>
Data Source:
```json
{{
    "metadata_info": [
        {{
            "name": "author",
            "type": "str",
            "description": "Author name"
        }},
        {{
            "name": "book_title",
            "type": "str",
            "description": "Book title"
        }},
        {{
            "name": "year",
            "type": "int",
            "description": "Year Published"
        }},
        {{
            "name": "pages",
            "type": "int",
            "description": "Number of pages"
        }},
        {{
            "name": "summary",
            "type": "str",
            "description": "A short summary of the book"
        }}
    ],
    "content_info": "Classic literature"
}}
```

User Query:
What are some books by Jane Austen published after 1813 that explore the theme of marriage for social standing?

Additional Instructions:
None

Structured Request:
```json
{{"query": "Books related to theme of marriage for social standing", "filters": [{{"key": "year", "value": "1813", "operator": ">"}}, {{"key": "author", "value": "Jane Austen", "operator": "=="}}], "top_k": null}}

```

<< Example 2. >>
Data Source:
```json
{info_str}
```

User Query:
{query_str}

Additional Instructions:
{additional_instructions}

Structured Request:
"""

In [30]:
prompt_tmpl = PromptTemplate(prompt_tmpl_str)

In [31]:
# Additional Instructions에 강화 프롬프트 포함
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.\
"""
prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)

In [32]:
retriever.update_prompts({"prompt": prompt_tmpl})

In [33]:
nodes = retriever.retrieve("Tell me about some books that are mafia-themed")
for node in nodes:
    print(node.text)
    print(node.metadata)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Books that are mafia-themed
Using query str: Books that are mafia-themed
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('theme', '==', 'mafia')]
Using filters: [('theme', '==', 'mafia')]
Using query str: Books that are mafia-themed
Using filters: [('theme', '==', 'mafia')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/met

In [34]:
nodes

[]

In [35]:
# 고쳐야 할 부분 고쳐보자
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.
Also, always convert the first character of the searching theme string to upper case.\
"""
prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)
retriever.update_prompts({"prompt": prompt_tmpl})

In [36]:
# 같은 쿼리 재시도
nodes = retriever.retrieve("Tell me about some books that are mafia-themed")
for node in nodes:
    print(node.text)
    print(node.metadata)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Books related to theme of Mafia
Using query str: Books related to theme of Mafia
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('theme', '==', 'Mafia')]
Using filters: [('theme', '==', 'Mafia')]
Using query str: Books related to theme of Mafia
Using filters: [('theme', '==', 'Mafia')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/col

In [37]:
#다른 문제 있었던 쿼리
nodes = retriever.retrieve("Tell me about some books that are Fiction")
for node in nodes:
    print(node.text)
    print(node.metadata)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Books that are Fiction
Using query str: Books that are Fiction
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('theme', '==', 'Fiction')]
Using filters: [('theme', '==', 'Fiction')]
Using query str: Books that are Fiction
Using filters: [('theme', '==', 'Fiction')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_e

In [38]:
# 테마를 명시하지 않아도 필터될수 있는 구조로 프롬프트 강화해보자
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.
Also, always convert the first character of the searching theme string to upper case.
Also, if the input is something like 'book that are 'something'', then it means something is the theme, and you should filter it accordingly.\
"""
prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)
retriever.update_prompts({"prompt": prompt_tmpl})

In [39]:
#업데이트된 프롬프트로 다시 시도
nodes = retriever.retrieve("Tell me about some books that are Fiction")
for node in nodes:
    print(node.text)
    print(node.metadata)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Books that are Fiction
Using query str: Books that are Fiction
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('theme', '==', 'Fiction')]
Using filters: [('theme', '==', 'Fiction')]
Using query str: Books that are Fiction
Using filters: [('theme', '==', 'Fiction')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://2dbee40a-5d76-4ab5-9f5d-a038db640bba.eu-west-2-0.aws.cloud.qdrant.io:6333/collections/metafilter_e